# Exploration d'une seule base de données

Ce notebook a pour vocation d'aider à rassembler les différents éléments qui pourraient être utile pour faire le benchmark entre Llama Index et Langchain. 

On commence d'abord par charger la base de données et explorer les différentes tables qui la compose.

In [3]:
# Mettre ici le nom de la base de données à explorer
import os

base_path = "../multispider/multispider/dataset/spider/database"
evaluation_path = "../multispider/multispider/model/multilingual/evaluation_fr.json"

dev_path = "../multispider/databases_dev.json"
train_path = "../multispider/databases_train.json"

## Liste complète des bases de données
# os.listdir(base_path)

In [15]:
# Sélectionnons la première base de données du jeu de dev
import json

with open(dev_path, "r") as f:
    dev_databases = json.load(f)

with open(train_path, "r") as f:
    train_databases = json.load(f)

work_database = dev_databases[0]
# work_database = "YOUR_DATABASE_NAME"
work_database

'wta_1'

## Chargement de la base de données

In [5]:
import sqlite3

# Créons une connexion SQL à la base de données
con = sqlite3.connect(os.path.join(base_path, work_database, work_database + ".sqlite"))

# Créons un curseur pour exécuter des requêtes SQL
cur = con.cursor()

In [7]:
# On peut maintenant lister les tables de la base de données
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()

In [8]:
tables

[('players',), ('matches',), ('rankings',)]

In [10]:
# On peut maintenant lister les colonnes de chaque table, puis le nombre de lignes dans chaque table et le type de données de chaque colonne

for table in tables:
    table_name = table[0]
    print(table_name)
    cur.execute(f"PRAGMA table_info({table_name});")
    columns = cur.fetchall()
    print(columns)
    cur.execute(f"SELECT COUNT(*) FROM {table_name};")
    n_rows = cur.fetchone()[0]
    print(f"Nombre de lignes : {n_rows}")
    print()

players
[(0, 'player_id', 'INT', 0, None, 1), (1, 'first_name', 'TEXT', 0, None, 0), (2, 'last_name', 'TEXT', 0, None, 0), (3, 'hand', 'TEXT', 0, None, 0), (4, 'birth_date', 'DATE', 0, None, 0), (5, 'country_code', 'TEXT', 0, None, 0)]
Nombre de lignes : 20662

matches
[(0, 'best_of', 'INT', 0, None, 0), (1, 'draw_size', 'INT', 0, None, 0), (2, 'loser_age', 'FLOAT', 0, None, 0), (3, 'loser_entry', 'TEXT', 0, None, 0), (4, 'loser_hand', 'TEXT', 0, None, 0), (5, 'loser_ht', 'INT', 0, None, 0), (6, 'loser_id', 'INT', 0, None, 0), (7, 'loser_ioc', 'TEXT', 0, None, 0), (8, 'loser_name', 'TEXT', 0, None, 0), (9, 'loser_rank', 'INT', 0, None, 0), (10, 'loser_rank_points', 'INT', 0, None, 0), (11, 'loser_seed', 'INT', 0, None, 0), (12, 'match_num', 'INT', 0, None, 0), (13, 'minutes', 'INT', 0, None, 0), (14, 'round', 'TEXT', 0, None, 0), (15, 'score', 'TEXT', 0, None, 0), (16, 'surface', 'TEXT', 0, None, 0), (17, 'tourney_date', 'DATE', 0, None, 0), (18, 'tourney_id', 'TEXT', 0, None, 0), (19,

In [13]:
# On affiche les 10 premières lignes de chaque table sous forme de pandas DataFrame
import pandas as pd


for table in tables:
    table_name = table[0]
    print(table_name)
    cur.execute(f"SELECT * FROM {table_name} LIMIT 10;")
    rows = cur.fetchall()
    columns = [description[0] for description in cur.description]
    df = pd.DataFrame(rows, columns=columns)
    print(df)
    print()

players
   player_id  first_name       last_name hand  birth_date country_code
0     200001     Martina          Hingis    R    19800930          SUI
1     200002     Mirjana           Lucic    R    19820309          CRO
2     200003     Justine           Henin    R    19820601          BEL
3     200004  Kerry Anne            Guse    R    19721204          AUS
4     200005      Jolene  Watanabe Giltz    R    19680831          USA
5     200006      Karina       Habsudova    R    19730802          SVK
6     200007     Silvija          Talaja    R    19780114          CRO
7     200008      Alicia           Molik    R    19810127          AUS
8     200009    Tamarine      Tanasugarn    R    19770524          THA
9     200010        Rita          Grande    R    19750323          ITA

matches
   best_of  draw_size  loser_age loser_entry loser_hand  loser_ht  loser_id  \
0        3          4  24.626968                      R       170    201474   
1        3          4  23.622177            

## Chargement des questions

On va charger les questions des fichiers dev_fr.json, examples_fr.json et train_fr.json s'ils existent pour la base de données sélectionnée. Nous allons ensuite afficher les questions et les requêtes SQL associées.

In [25]:
def print_questions(questions):
    for q in questions:
        print(q["question"])
        print(q["query"])
        print()
    print()

def print_query_questions(work_database, base_path):
    if work_database in dev_databases:
        with open(os.path.join(base_path, work_database, "dev_fr.json"), "r", encoding="utf-8") as f:
            dev_fr = json.load(f)
        print("Questions dev_fr.json")
        print_questions(dev_fr)
        
        with open(os.path.join(base_path, work_database, "examples_fr.json"), "r", encoding="utf-8") as f:
            examples = json.load(f)
        print("Questions examples.json")
        print_questions(examples)
        
    elif work_database in train_databases:
        with open(os.path.join(base_path, work_database, "examples.json"), "r", encoding="utf-8") as f:
            examples = json.load(f)
        print("Questions examples.json")
        print_questions(examples)
        
        with open(os.path.join(base_path, work_database, "train_fr.json"), "r", encoding="utf-8") as f:
            train_fr = json.load(f)
        print("Questions train_fr.json")
        print_questions(train_fr)
    
print_query_questions(work_database, base_path)

Questions dev_fr.json
Trouvez le nombre total de joueurs.
SELECT count(*) FROM players

Combien de joueurs y a-t-il?
SELECT count(*) FROM players

Trouvez le nombre total de matchs.
SELECT count(*) FROM matches

Comptez le nombre de matchs.
SELECT count(*) FROM matches

Indiquez le prénom et la date de naissance de tous les joueurs du pays avec le code USA.
SELECT first_name ,  birth_date FROM players WHERE country_code  =  'USA'

Quels sont les prénoms et les dates de naissance des joueurs des États-Unis?
SELECT first_name ,  birth_date FROM players WHERE country_code  =  'USA'

Trouvez l'âge moyen des perdants et des gagnants de tous les matchs.
SELECT avg(loser_age) ,  avg(winner_age) FROM matches

Quel est l'âge moyen des perdants et des gagnants d'un match à l'autre?
SELECT avg(loser_age) ,  avg(winner_age) FROM matches

Trouvez le classement moyen des gagnants dans tous les matchs.
SELECT avg(winner_rank) FROM matches

Quel est le classement moyen des gagnants dans tous les match

## Utilisation du pipeline de requêtes SQL pour tester les questions et les performances

In [ ]:
### TO DO

## 